# Optimizer:  
tf.train.GradientDescentOptimizer()    
tf.train.AdadeltaOptimizer()    
tf.train.AdagradOptimizer()     
tf.train.AdagradDAOptimizer()    
tf.train.MomentumOptimizer()   
tf.train.AdamOptimizer()   
tf.FtrlOptimizer()   
tf.train.ProximalGradientDescentOptimizer()     
tf.train.ProximalAdagradOptimizer()    
tf.train.RMSPropOptimizer()    
  
# 各种优化器对比：
标准梯度下降法：标准梯度下降先计算所有样本汇总误差，然后根据总误差来更新权值。  
随机梯度下降法：随机梯度下降随机抽取一个样本来计算误差，然后更新权值  
批量梯度下降法：批量梯度下降算是一种折中的方案，从总样本中选取一个批次（比如一共有1000的样本，随机选取100个样本作为一个batch）,然后计算这个batch的总误差，根据总误差来更新权值。   
SGD（容易陷入局部最优，收敛速度很慢）  
Momentum  
NAG  
Adagrad  
RMSprop  
Adadelta（建议使用，模型收敛速度快）  
Adam  

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

# 每个批次的大小
batch_size = 100  # 以矩阵的形式放进去
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)
lr = tf.Variable(0.001,dtype=tf.float32)

# 创建一个复杂的神经网络
W1 = tf.Variable(tf.truncated_normal([784,500],stddev=0.1))  # 截断的正态分布
b1 = tf.Variable(tf.zeros([500])+0.1)
L1 = tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop = tf.nn.dropout(L1,keep_prob)

W2 = tf.Variable(tf.truncated_normal([500,300],stddev=0.1))  # 截断的正态分布
b2 = tf.Variable(tf.zeros([300])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
L2_drop = tf.nn.dropout(L2,keep_prob)

W3 = tf.Variable(tf.truncated_normal([300,10],stddev=0.1))  # 截断的正态分布
b3 = tf.Variable(tf.zeros([10])+0.1)
prediction = tf.nn.softmax(tf.matmul(L2_drop,W3)+b3)


# # 情形1：二次代价函数
# loss = tf.reduce_mean(tf.square(y-prediction))
# 情形2：对数似然代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

# 使用梯度下降算法
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))  # tf.argmax(prediction,1) 返回概率最大的位置（标签）
# argmax返回一维张量中最大的值所在的位置

# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(51):  # 迭代
        sess.run(tf.assign(lr, 0.001 * (0.95 ** epoch)))
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0})
#             # Dropout防止过拟合
#             sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        learning_rate = sess.run(lr) 
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print('Iter ' + str(epoch) + ',Testing Accuracy ' + str(test_acc)+ ',Learning Rate ' + str(learning_rate))    

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter 0,Testing Accuracy 0.952,Learning Rate 0.001
Iter 1,Testing Accuracy 0.9632,Learning Rate 0.00095
Iter 2,Testing Accuracy 0.9666,Learning Rate 0.0009025
Iter 3,Testing Accuracy 0.9735,Learning Rate 0.000857375
Iter 4,Testing Accuracy 0.9738,Learning Rate 0.00081450626
Iter 5,Testing Accuracy 0.9742,Learning Rate 0.0007737809
Iter 6,Testing Accuracy 0.9757,Learning Rate 0.0007350919
Iter 7,Testing Accuracy 0.975,Learning Rate 0.0006983373
Iter 8,Testing Accuracy 0.9785,Learning Rate 0.0006634204
Iter 9,Testing Accuracy 0.9773,Learning Rate 0.0006302494
Iter 10,Testing Accuracy 0.9775,Learning Rate 0.0005987369
Iter 11,Testing Accuracy 0.9777,Learning Rate 0.0005688001
Iter 12,Testing Accuracy 0.9796,Learning Rate 0.0005403601
Iter 13,Testing Accuracy 0.9796,Learning Rate 0.0005133421
Iter 14